In [13]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

def Q_learning(tabuleiro):
    N_ACOES = 4  # 0: cima, 1: baixo, 2: esquerda, 3: direita
    acoes = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Parâmetros ajustados
    learning_rate = 0.2
    discount_factor = 0.95
    exploration_initial = 0.5
    exploration_final = 0.01
    epocas = 15000
    max_passos = 200

    # Inicialização da Q-table com valores negativos para ações inválidas
    Q = np.full((16, 2, 2, 2, N_ACOES), -np.inf)

    # Preenche valores iniciais apenas para ações válidas
    for pos in range(16):
        i, j = pos//4 + 1, pos%4 + 1  # Converte posição linear para (i,j)
        for ouro in range(2):
            for flecha in range(2):
                for monstro in range(2):
                    for acao in range(N_ACOES):
                        ni, nj = i + acoes[acao][0], j + acoes[acao][1]
                        if 1 <= ni <= 4 and 1 <= nj <= 4 and tabuleiro[ni][nj]['parede'] == 0:
                            Q[pos, ouro, flecha, monstro, acao] = 0  # Valor inicial neutro

    recompensas_epocas = []

    for epoca in range(epocas):
        exploration_prob = exploration_final + (exploration_initial - exploration_final) * \
                         np.exp(-epoca / (epocas * 0.2))

        pos = (1, 1)
        estado_pos = tabuleiro[pos[0]][pos[1]]['posicao']
        ouro_coletado = False
        flecha = 1
        monstro_vivo = any(tabuleiro[i][j]['monstro'] == 1
                          for i in range(1,5) for j in range(1,5))

        recompensa_epoca = 0
        terminado = False

        for passo in range(max_passos):
            # Apenas ações válidas
            acoes_validas = []
            for acao in range(N_ACOES):
                ni, nj = pos[0] + acoes[acao][0], pos[1] + acoes[acao][1]
                if 1 <= ni <= 4 and 1 <= nj <= 4 and tabuleiro[ni][nj]['parede'] == 0:
                    acoes_validas.append(acao)

            if not acoes_validas:
                break

            # Escolha de ação apenas entre as válidas
            if np.random.random() < exploration_prob:
                acao = np.random.choice(acoes_validas)
            else:
                q_validas = [Q[estado_pos, int(ouro_coletado), flecha, int(monstro_vivo)][a]
                           for a in acoes_validas]
                acao = acoes_validas[np.argmax(q_validas)]

            # Executar ação
            nova_pos = (pos[0] + acoes[acao][0], pos[1] + acoes[acao][1])
            recompensa = -1  # custo padrão por movimento

            # Verificar consequências
            if tabuleiro[nova_pos[0]][nova_pos[1]]['buraco'] == 1:
                recompensa = -100
                terminado = True

            elif tabuleiro[nova_pos[0]][nova_pos[1]]['monstro'] == 1 and monstro_vivo:
                recompensa = -100
                terminado = True

            elif tabuleiro[nova_pos[0]][nova_pos[1]]['cheiro'] == 1 and flecha == 1 and monstro_vivo:
                mi, mj = next((i,j) for i in range(1,5) for j in range(1,5)
                             if tabuleiro[i][j]['monstro'] == 1)

                if (abs(nova_pos[0] - mi) + abs(nova_pos[1] - mj)) == 1:
                    recompensa = 50
                    monstro_vivo = False
                    flecha = 0
                else:
                    recompensa = -20
                    flecha = 0

            elif tabuleiro[nova_pos[0]][nova_pos[1]]['brilho'] == 1 and not ouro_coletado:
                recompensa = 50
                ouro_coletado = True

            elif nova_pos == (1, 1) and ouro_coletado and (not monstro_vivo or flecha == 0):
                recompensa = 100
                terminado = True

            # Atualização Q-table
            novo_estado_pos = tabuleiro[nova_pos[0]][nova_pos[1]]['posicao']

            current_q = Q[estado_pos, int(ouro_coletado), flecha, int(monstro_vivo)][acao]

            if terminado:
                max_future_q = 0
            else:
                max_future_q = np.max(Q[novo_estado_pos, int(ouro_coletado), flecha, int(monstro_vivo)])

            Q[estado_pos, int(ouro_coletado), flecha, int(monstro_vivo)][acao] = \
                current_q + learning_rate * (recompensa + discount_factor * max_future_q - current_q)

            # Atualizar estado
            pos = nova_pos
            estado_pos = novo_estado_pos
            recompensa_epoca += recompensa

            if terminado:
                break

        recompensas_epocas.append(recompensa_epoca)

    # Extrair política ótima
    politica_otima = np.zeros((16, 2, 2, 2), dtype=int)
    for pos in range(16):
        for ouro in range(2):
            for flecha in range(2):
                for monstro in range(2):
                    # Seleciona apenas entre ações válidas
                    i, j = pos//4 + 1, pos%4 + 1
                    acoes_validas = []
                    for acao in range(N_ACOES):
                        ni, nj = i + acoes[acao][0], j + acoes[acao][1]
                        if 1 <= ni <= 4 and 1 <= nj <= 4 and tabuleiro[ni][nj]['parede'] == 0:
                            acoes_validas.append(acao)

                    if acoes_validas:
                        q_validas = [Q[pos, ouro, flecha, monstro][a] for a in acoes_validas]
                        politica_otima[pos, ouro, flecha, monstro] = acoes_validas[np.argmax(q_validas)]

    return Q, politica_otima

def simula_politica_otima(tabuleiro, Q, politica_otima):
    pos = (1, 1)
    estado_pos = tabuleiro[pos[0]][pos[1]]['posicao']
    acoes = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # cima, baixo, esquerda, direita
    passos = [pos]
    ouro_coletado = False
    flecha = 1
    monstro_vivo = any(tabuleiro[i][j]['monstro'] == 1
                      for i in range(1,5) for j in range(1,5))
    pontos = 0

    print("\n--- Execução da política ótima ---")

    for _ in range(100):
        # Obter ação da política ótima considerando estado completo
        acao = politica_otima[estado_pos, int(ouro_coletado), flecha, int(monstro_vivo)]
        delta = acoes[acao]
        nova_pos = (pos[0] + delta[0], pos[1] + delta[1])

        if not (1 <= nova_pos[0] <= 4 and 1 <= nova_pos[1] <= 4):
            print(f"Tentou sair do mapa indo para {nova_pos}, ignorando...")
            pontos -= 5
            continue

        print(f"Agente se move de {pos} para {nova_pos}")
        pos = nova_pos
        passos.append(pos)
        estado_pos = tabuleiro[pos[0]][pos[1]]['posicao']

        # Checagem de morte
        if tabuleiro[pos[0]][pos[1]]['buraco'] == 1:
            print("Caiu no buraco! Fim do jogo.")
            pontos -= 100
            break

        if tabuleiro[pos[0]][pos[1]]['monstro'] == 1 and monstro_vivo:
            print("Foi comido pelo monstro! Fim do jogo.")
            pontos -= 100
            break

        # Atirar a flecha se sentir cheiro e ainda tiver
        if tabuleiro[pos[0]][pos[1]]['cheiro'] == 1 and flecha == 1 and monstro_vivo:
            mi, mj = next((i,j) for i in range(1,5) for j in range(1,5)
                         if tabuleiro[i][j]['monstro'] == 1)

            if (abs(pos[0] - mi) + abs(pos[1] - mj)) == 1:  # Adjacente
                print("Atirou a flecha e matou o monstro!")
                pontos += 50
                monstro_vivo = False
                tabuleiro[mi][mj]['monstro'] = 0
            else:
                print("Atirou a flecha e errou.")
                pontos -= 20
            flecha = 0

        # Ouro
        if tabuleiro[pos[0]][pos[1]]['brilho'] == 1 and not ouro_coletado:
            print("Pegou o ouro!")
            pontos += 50
            ouro_coletado = True
            tabuleiro[pos[0]][pos[1]]['brilho'] = 0

        # Volta ao início com ouro
        if pos == (1, 1) and ouro_coletado and (not monstro_vivo or flecha == 0):
            print("Retornou à posição inicial com o ouro! Vitória!")
            pontos += 100
            break

        pontos -= 1  # custo de movimentação

    print(f"Pontuação final: {pontos}")
    print(f"Passos percorridos: {passos}")

def simulador(tabuleiro,flecha):
    passos = []
    passos_corretos=[]
    ouro_pos = (0,0)
    monstro_pos=(0,0)
    monstro=0
    pontos = 0
    ordem=[]
    i=0
    ouro=0
    for i in range(1, 5):
        for j in range(1, 5):
            if tabuleiro [i][j]['monstro']==1:
              monstro_pos=(i,j)
    for i in range(1, 5):
        for j in range(1, 5):
          if tabuleiro[i][j]['caminho']=='certo':
            current=(i,j)
            if tabuleiro[i][j]['brilho'] == 1:
                ouro_pos=(i,j)
                while isinstance(tabuleiro[i][j]['antecessor'], tuple):
                    if flecha<1:
                      if (1 <= i + 1 < 5 and 1 <= j < 5 and tabuleiro[i+1][j]['monstro'] == 1) or (1 <= i < 5 and 1 <= j + 1 < 5 and tabuleiro[i][j+1]['monstro'] == 1):
                        passos.append((i, j))
                        break
                    passos.append((i, j))
                    ii = tabuleiro[i][j]["antecessor"][0]
                    jj = tabuleiro[i][j]["antecessor"][1]
                    i = ii
                    j = jj
                passos_corretos+=reversed(passos)
                passos=[]
                (i,j)=current
                BFS(tabuleiro,(i,j),pontos)
            elif ((1 <= i + 1 < 5 and 1 <= j < 5 and tabuleiro[i+1][j]['monstro'] == 1) or (1 <= i < 5 and 1 <= j + 1 < 5 and tabuleiro[i][j+1]['monstro'] == 1)) and flecha==1:
                pontos += 50
                flecha+=-1
                while isinstance(tabuleiro[i][j]['antecessor'], tuple):
                    if ouro_pos!=(0,0):
                      if i==ouro_pos[0] and j==ouro_pos[1]:
                        passos.append((i, j))
                        break
                    passos.append((i, j))
                    ii = tabuleiro[i][j]["antecessor"][0]
                    jj = tabuleiro[i][j]["antecessor"][1]
                    i = ii
                    j = jj
                passos_corretos+=reversed(passos)
                passos=[]
                (i,j)=current
                BFS(tabuleiro,(i,j),pontos)
    if passos_corretos:
        for i,j in passos_corretos:
            print(f'O simulador se move para a casa {i},{j}')
            if tabuleiro[i][j]['brilho'] == 1 and ouro==0:
                print('O simulador achou o ouro!')
                ouro=1
            if ((1 <= i + 1 < 5 and 1 <= j < 5 and tabuleiro[i+1][j]['monstro'] == 1) or (1 <= i < 5 and 1 <= j + 1 < 5 and tabuleiro[i][j+1]['monstro'] == 1)) and monstro==0:
                print('O simulador atirou a flecha no monstro!')
                monstro=1
        lista = len(passos_corretos)
        pontos += -lista
        volta_inicio(tabuleiro, passos_corretos[lista-1], pontos, ouro)
    else:
      print('Lista vazia')

def volta_inicio(tabuleiro, v, pontos, ouro):
  volta=0
  BFS(tabuleiro,(1,1),pontos)
  while tabuleiro[v[0]][v[1]]['antecessor'] != 'NULL':
    volta+=1
    print(f'O simulador volta para a posição {tabuleiro[v[0]][v[1]]["antecessor"]}')
    v=tabuleiro[v[0]][v[1]]["antecessor"]
  pontos+=-volta
  if ouro==1:
    pontos+=50
  print(f"O simulador terminou o jogo com: {pontos}")

def BFS(tabuleiro, inicio, pontos):
  for i in range(1, 5):
    for j in range(1, 5):
      tabuleiro[i][j]['cor'] = 'branco'
      tabuleiro[i][j]['distancia'] = 0
      tabuleiro[i][j]['antecessor'] = 'NULL'
      tabuleiro[i][j]['caminho'] = 'errado'
  tabuleiro[inicio[0]][inicio[1]]['cor'] = 'cinza'
  fila = [inicio]
  while fila:
    u = fila.pop(0)
    if tabuleiro[u[0]][u[1]]['buraco']==0:
      for v in [(u[0]-1, u[1]), (u[0]+1, u[1]), (u[0], u[1]-1), (u[0], u[1]+1)]:
        if tabuleiro[v[0]][v[1]]['parede']==1:
          continue
        if tabuleiro[v[0]][v[1]]['cor'] == 'branco':
          if tabuleiro[v[0]][v[1]]['monstro']==0 and tabuleiro[v[0]][v[1]]['buraco']==0:
            tabuleiro[v[0]][v[1]]['cor'] = 'cinza'
            tabuleiro[v[0]][v[1]]['distancia'] = tabuleiro[u[0]][u[1]]['distancia'] + 1
            tabuleiro[v[0]][v[1]]['antecessor'] = u
            tabuleiro[v[0]][v[1]]['caminho'] = 'certo'
            fila.append(v)

      tabuleiro[u[0]][u[1]]['cor'] = 'preto'
    else:
      print('Buraco entrou na fila')
  return tabuleiro

def verifica(jogador,pontos):
  if tabuleiro[jogador[0]][jogador[1]]['monstro'] == 1:
    print(f'Você está na posição {jogador} e encontrou o monstro!')
    return 'morto', pontos-100

  if tabuleiro[jogador[0]][jogador[1]]['buraco'] == 1:
    print(f'Você está na posição {jogador} e caiu em um buraco!')
    return 'morto', pontos - 100

  if tabuleiro[jogador[0]][jogador[1]]['brilho'] == 1:
    print(f'Você está na posição {jogador} e encontrou o ouro!')

  if tabuleiro[jogador[0]][jogador[1]]['parede'] == 1:
    print(f'Você está na posição {jogador} e bateu em uma parede!')

  if tabuleiro[jogador[0]][jogador[1]]['cheiro'] == 1:
    print(f'Você está na posição {jogador} e sentiu um fedor!')

  if tabuleiro[jogador[0]][jogador[1]]['brisa'] == 1:
    print(f'Você está na posição {jogador} e sentiu uma brisa!')

  if tabuleiro[jogador[0]][jogador[1]]['brisa'] == 0 and tabuleiro[jogador[0]][jogador[1]]['cheiro'] == 0 and tabuleiro[jogador[0]][jogador[1]]['parede'] == 0 and tabuleiro[jogador[0]][jogador[1]]['brilho'] == 0 and tabuleiro[jogador[0]][jogador[1]]['monstro'] == 0 and tabuleiro[jogador[0]][jogador[1]]['buraco'] == 0:
    print(f'Você está na posição {jogador} e não percebe nada.')

  return jogador, pontos


def verifica2(flecha, pontos, f):
  if tabuleiro[flecha[0]][flecha[1]]['monstro'] == 1:
    print('O Wumpus é morto e solta um grito. Você recebeu 50 pontos!')
    pontos+=50
    tabuleiro[flecha[0]][flecha[1]]['monstro'] = 0
    f+=-1
  else:
    f+=-1
  return f


def aleatorio(x):
  valor_aleatorio = random.randint(1, 15)
  for i in range(1,16):
    if valor_aleatorio==i:
      for j in range(1,5):
        for k in range(1,5):
          if j==1 and k==1:
            continue
          if tabuleiro[j][k]['posicao']==i:
            if x=='brilho' and (tabuleiro[j][k]['brilho']==1 or tabuleiro[j][k]['monstro']==1 or tabuleiro[j][k]['buraco']==1):
              aleatorio('brilho')
              return
            elif x=='buraco' and (tabuleiro[j][k]['brilho']==1 or tabuleiro[j][k]['monstro']==1 or tabuleiro[j][k]['buraco']==1):
              aleatorio('buraco')
              return
            elif x=='monstro' and (tabuleiro[j][k]['brilho']==1 or tabuleiro[j][k]['monstro']==1 or tabuleiro[j][k]['buraco']==1):
              aleatorio('monstro')
              return
            else:
              tabuleiro[j][k][x]=1
              print(f'O {x} está na posição {j},{k}')
              if x == 'buraco':
                if j > 1:
                  tabuleiro[j-1][k]['brisa'] = 1
                if j < 4:
                  tabuleiro[j+1][k]['brisa'] = 1
                if k > 1:
                  tabuleiro[j][k-1]['brisa'] = 1
                if k < 4:
                  tabuleiro[j][k+1]['brisa'] = 1
              elif x == 'monstro':
                if j > 1:
                  tabuleiro[j-1][k]['cheiro'] = 1
                if j < 4:
                  tabuleiro[j+1][k]['cheiro'] = 1
                if k > 1:
                  tabuleiro[j][k-1]['cheiro'] = 1
                if k < 4:
                  tabuleiro[j][k+1]['cheiro'] = 1
              return

def main():
  f=1
  pontos=0
  i=0

  for j in range(1,5):
    for k in range(1,5):
      caracteristicas={'linha':0,'coluna':0,'monstro':0,'buraco':0,'brilho':0,'parede':0, 'cheiro':0, 'brisa':0, 'posicao':i, 'cor':'branco', 'distancia':0, 'antecessor': 'NULL', 'caminho':'errado'}
      tabuleiro[j][k]=caracteristicas
      tabuleiro[j][k]['linha']=j
      tabuleiro[j][k]['coluna']=k
      tabuleiro[j][k]['posicao']=i
      i+=1
      if i==36:
        break

  aleatorio('monstro')
  aleatorio('buraco')
  aleatorio('buraco')
  aleatorio('buraco')
  aleatorio('brilho')

  for i in range(6):
    for j in range(6):
      if i==0 or j==0 or i==5 or j==5:
        tabuleiro[i][j]['parede']=1

  jogador=(1,1)
  verifica(jogador,pontos)

  if jogador=='morto':
    print(f'Você morreu com {pontos} pontos!')
    return 0

  print('1- Jogar')
  print('2- Simulador com busca em largura')
  opcao=input('3- Simulador com aprendizado por reforço')
  if opcao=='1':
    while True:
      menu=input('1-move para direção, 2-atira_flecha, 3-pega_ouro, 4-escala_caverna ')
      if menu=='1':
        x = int(input('Digite a nova linha (1-4): '))
        y = int(input('Digite a nova coluna (1-4): '))
        nova_posicao = (x, y)
        if nova_posicao!=(jogador[0]+1,jogador[1]) and nova_posicao!=(jogador[0],jogador[1]+1) and nova_posicao!=(jogador[0]-1,jogador[1]) and nova_posicao!=(jogador[0],jogador[1]-1):
          print('Você só pode se mover uma casa.')
        else:
          jogador=nova_posicao
          jogador, pontos = verifica(jogador,pontos)
          if jogador=='morto':
            print(f'Você morreu com {pontos} pontos!')
            break
      elif menu=='2':
        if f==1:
          x = int(input('Digite a linha (1-4): '))
          y = int(input('Digite a coluna (1-4): '))
          flecha = (x, y)
          verifica2(flecha, pontos,f)
      elif menu=='3':
        if tabuleiro[jogador[0]][jogador[1]]['brilho'] == 1:
          print('Você pegou o ouro, para ganhar 50 pontos volte para a posição 0,0 e saia da caverna')
          tabuleiro[jogador[0]][jogador[1]]['brilho'] = 0
      elif menu=='4':
        if jogador!=(1,1):
          print('Você não está na posição inicial, volte para a posição 0,0 para escalar')
        else:
          pontos+=50
          print(f'Parabéns, você saiu da caverna com {pontos} pontos!')
          break
      else:
        print('Opção inválida')
      pontos+=-1

  if opcao=='2':
    BFS(tabuleiro,(1,1),pontos)
    simulador(tabuleiro,f)

  if opcao=='3':
    Q, politica_otima = Q_learning(tabuleiro)
    simula_politica_otima(tabuleiro, Q, politica_otima)

tabuleiro = [[{} for _ in range(6)] for _ in range(6)]
main()

O monstro está na posição 4,3
O buraco está na posição 2,4
O buraco está na posição 3,1
O buraco está na posição 1,2
O brilho está na posição 3,2
Você está na posição (1, 1) e sentiu uma brisa!
1- Jogar
2- Simulador com busca em largura
3- Simulador com aprendizado por reforço3

--- Execução da política ótima ---
Agente se move de (1, 1) para (2, 1)
Agente se move de (2, 1) para (2, 2)
Agente se move de (2, 2) para (3, 2)
Pegou o ouro!
Agente se move de (3, 2) para (4, 2)
Atirou a flecha e matou o monstro!
Agente se move de (4, 2) para (3, 2)
Agente se move de (3, 2) para (2, 2)
Agente se move de (2, 2) para (2, 1)
Agente se move de (2, 1) para (1, 1)
Retornou à posição inicial com o ouro! Vitória!
Pontuação final: 193
Passos percorridos: [(1, 1), (2, 1), (2, 2), (3, 2), (4, 2), (3, 2), (2, 2), (2, 1), (1, 1)]


#Referências
Consultei material de Algoritmos em Bioinformatica, que fiz com o Thiago Martini.